In [1]:
import sys
print(sys.executable)

C:\Users\delta\OneDrive\Desktop\Github\customer-churn-analytics-pipeline\venv\Scripts\python.exe


In [14]:
import pandas as pd


In [15]:
df = pd.read_csv(r"../data/raw_telco.csv")

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [16]:
df.shape

(7043, 21)

## We find out datatype of each column

In [17]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   str    
 1   gender            7043 non-null   str    
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   str    
 4   Dependents        7043 non-null   str    
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   str    
 7   MultipleLines     7043 non-null   str    
 8   InternetService   7043 non-null   str    
 9   OnlineSecurity    7043 non-null   str    
 10  OnlineBackup      7043 non-null   str    
 11  DeviceProtection  7043 non-null   str    
 12  TechSupport       7043 non-null   str    
 13  StreamingTV       7043 non-null   str    
 14  StreamingMovies   7043 non-null   str    
 15  Contract          7043 non-null   str    
 16  PaperlessBilling  7043 non-null   str    
 17  Paymen

## Converting "TotalCharges" type from str to numeric

In [20]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

df['TotalCharges'].dtype

dtype('float64')

## Checking missing/null values in columns 

In [21]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

## Removing Rows with Missing Values 

In [24]:
df = df.dropna()
df.shape

(7032, 21)

## Creating Business metrics

In [32]:
df['Churn_Flag'] = df['Churn'].str.strip()

df['Churn'].unique()

<StringArray>
['No', 'Yes']
Length: 2, dtype: str

In [33]:
df['Churn_Flag'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

df[['Churn', 'Churn_Flag']].head()

,Churn,Churn_Flag
0,No,0
1,No,0
2,Yes,1
3,No,0
4,Yes,1


## Create Revenue Loss 

In [34]:
df['Revenue_Loss'] = df['MonthlyCharges'] * df['Churn_Flag']

df[['MonthlyCharges', 'Churn_Flag', 'Revenue_Loss']].head()


,MonthlyCharges,Churn_Flag,Revenue_Loss
0,29.85,0,0.00
1,56.95,0,0.00
2,53.85,1,53.85
3,42.30,0,0.00
4,70.70,1,70.70


## Create Customer Lifetime Value 
### (Total revenue contributes by each customer)

In [35]:
df['CLV'] = df['MonthlyCharges'] * df['tenure']

df[['tenure', 'MonthlyCharges', 'CLV']].head()


,tenure,MonthlyCharges,CLV
0,1,29.85,29.85
1,34,56.95,1936.30
2,2,53.85,107.70
3,45,42.30,1903.50
4,2,70.70,141.40


In [36]:
df['Churn'].unique()

<StringArray>
['No', 'Yes']
Length: 2, dtype: str

### Churned Customers

In [37]:
df['Churn_Flag'].sum()


np.int64(1869)

### Monthly Revenue Lost 

In [39]:

df['Revenue_Loss'].sum()

np.float64(139130.84999999998)

## Calculating Churn Rate

In [40]:
total_customers = len(df)
churned_customers = df['Churn_Flag'].sum()

churn_rate = (churned_customers / total_customers) * 100

churn_rate

np.float64(26.578498293515356)

### Identified 26% customer churn rate resulting in $139K monthly revenue leakage

## Which Contract type causes highest churn 


In [41]:
df.groupby('Contract')['Churn_Flag'].mean()*100

Contract
Month-to-month    42.709677
One year          11.277174
Two year           2.848665
Name: Churn_Flag, dtype: float64

## We can see that month-to-month contracts are 15x more likely to churn than 2 year contracts


### Now we figure out how to reduce this revenue loss by simulating 

In [45]:
month_customers = df[df['Contract'] == 'Month-to-month']

total_month_customers = len(month_customers)
month_churned = month_customers['Churn_Flag'].sum()

print("Total Month-to-Month Customers: ", total_month_customers)
print("Churned Month-to-Month Customers: ", month_churned)

churn_rate_check = (month_churned/total_month_customers) * 100

print("Churn Rate for monthly contract : ", churn_rate_check)

Total Month-to-Month Customers:  3875
Churned Month-to-Month Customers:  1655
Churn Rate for monthly contract :  42.70967741935484


### Calculating Revenue from Month-to-Month Customers


In [47]:
month_revenue_loss = month_customers['Revenue_Loss'].sum()

print("Revenue generated from Month-to-Month Customers: ",month_revenue_loss)

Revenue generated from Month-to-Month Customers:  120847.1


## Lets Simulate converting 50% month-to-month customers to 1-Year Contracts


In [49]:
current_churn_rate = 0.427 #(42.7%)

one_year_churn_rate = 0.112 #(11.2%)

converted_customers = total_month_customers * 0.5

expected_churn_before = converted_customers * current_churn_rate

expected_churn_after = converted_customers * one_year_churn_rate

customers_saved = expected_churn_before - expected_churn_after

print("Retained Customers after converting 50% to 1 year contracts: ", customers_saved)

Retained Customers after converting 50% to 1 year contracts:  610.3125


### After Retaining 610 customers, we now estimate the Revenue Saved


In [50]:
avg_monthly_charge = month_customers['MonthlyCharges'].mean()

print("Average Monthly Charges: ", avg_monthly_charge)

Average Monthly Charges:  66.39849032258066


In [51]:
revenue_saved = customers_saved * avg_monthly_charge

print("Revenue Saved: ", revenue_saved)

Revenue Saved:  40523.82862500001


In [52]:
df.to_csv("../data/cleaned_telco.csv", index=False)

### End Result : 
### Built end-to-end customer churn analytics pipeline (Python, SQL Server, Power BI) analyzing 7K+ records; identified month-to-month contracts as primary churn driver (42.7%) and modeled retention strategy projecting $40K monthly revenue recovery.